In [ ]:
!pip install datasets transformers huggingface_hub

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

train_data = pd.read_csv("train.csv",on_bad_lines='skip')
test_data = pd.read_csv("test.csv", on_bad_lines='skip')

train_data['prompt'] = train_data['prompt'].str.replace('_comma_', ',')
test_data['prompt'] = test_data['prompt'].str.replace('_comma_', ',')

label_mapping = {label: i for i, label in enumerate(train_data['context'].unique())}
train_data['label'] = train_data['context'].map(label_mapping)
test_data['label'] = test_data['context'].map(label_mapping)

train_dataset = Dataset.from_pandas(train_data[['prompt', 'label']][:20000])
test_dataset = Dataset.from_pandas(test_data[['prompt', 'label']])

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["prompt"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_mapping))

training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained("./Sentiment_Model")
tokenizer.save_pretrained("./Sentiment_Model")

metrics = trainer.evaluate()
print(metrics)


c:\Users\dbahe\Downloads\ML_Model_End_To_End\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

train_data = pd.read_csv("Datasets/train.csv",on_bad_lines='skip')
test_data = pd.read_csv("Datasets/test.csv", on_bad_lines='skip')

train_data['prompt'] = train_data['prompt'].str.replace('_comma_', ',')
test_data['prompt'] = test_data['prompt'].str.replace('_comma_', ',')

label_mapping = {label: i for i, label in enumerate(train_data['context'].unique())}

model_name_or_path = "dhruvbcodes/Sentiment_Model"  
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)

input_text = ["I had a argument with my boss, it was his fault.",
              "I received concert tickets for Christmas!"]

inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_labels = [reverse_label_mapping[label_id] for label_id in predictions.tolist()]

print(predicted_labels)

['angry', 'joyful']


In [27]:
from pydantic import BaseModel
from typing import Optional

class SentimentRequest(BaseModel):
    text: str 
    location: Optional[str] = None


obj = SentimentRequest(text="i am fine")
print(obj.location)

print(obj.location if obj.location is not None else 'mumbai')

None
mumbai
